In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Definir transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargar el conjunto de datos MNIST
train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, transform=transform)

# Definir el tamaño del lote y crear los iteradores de datos
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 11577208.21it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 5776894.17it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5277388.82it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [11]:
# Verificar si hay una GPU disponible y establecer el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
import torch
import torch.nn as nn

class MyConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(MyConvolution, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size)).to(device)
        self.bias = nn.Parameter(torch.zeros(out_channels)).to(device)
        self.kernel_size = kernel_size

    def forward(self, x):
        batch_size, in_channels, height, width = x.size()
        out_channels = self.weight.size(0)

        output_height = height - self.kernel_size + 1
        output_width = width - self.kernel_size + 1

        output = torch.zeros(batch_size, out_channels, output_height, output_width).to(device)

        # Realizar la convolución
        for i in range(output_height):
            for j in range(output_width):
                receptive_field = x[:, :, i:i+self.kernel_size, j:j+self.kernel_size].to(device)
                #necesitamos el unsqueeze de receptive field para añadir una dimensión en la posición (1) de receptive field
                #esto porque es como si en donde tenemos el 1 se "repitiera" la información
                #para cada imagen del batch queremos que se multiplique por el mismo kernel, por eso se debe "repetir"
                #además en este caso tenemos que hacer coincidir la dimensión de 'in_channels' de receptive_field y de weights
                #la suma se hace sobre las dimensiones 2,3,4 porque la convolución suma sobre los canales de entrada y después a lo largo
                #del kernel size
                conv = torch.sum(receptive_field.unsqueeze(1) * self.weight.unsqueeze(0), dim=(2, 3, 4)) + self.bias.unsqueeze(0)
                output[:, :, i, j] = conv.squeeze()

        return output

# Ejemplo de uso
input = torch.randn(16, 3, 32, 32)  # Ejemplo de tensor de entrada con un lote de tamaño 16
conv = MyConvolution(3, 64, 3)  # Crear una instancia de tu función de convolución personalizada
output = conv(input)  # Realizar la convolución

print(output.size())  # Imprimir el tamaño de salida


torch.Size([16, 64, 30, 30])


In [ ]:


# Definir el modelo de la red neuronal
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = MyConvolution(1, 3, kernel_size=3)
        self.conv2 = MyConvolution(3, 6, kernel_size=3)
        self.fc1 = nn.Linear(6* 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 6 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Crear una instancia del modelo
model = Net().to(device)


# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Entrenamiento del modelo
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # Calcular las salidas y la pérdida
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Realizar la retropropagación y la optimización
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Epoch: 1, Batch: 100, Loss: 1.292
Epoch: 1, Batch: 200, Loss: 0.394
Epoch: 1, Batch: 300, Loss: 0.350
Epoch: 1, Batch: 400, Loss: 0.315


KeyboardInterrupt: 